### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [2]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function parabola(x)
    a = 0.13625841750841752
    b = 0.189709595959596
    c = 0.22
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [3]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [4]:
startI = -1.75
endI = -1.705
lengthI = 200

rangeI = range( startI, endI, length = lengthI )

step_to_left = rangeI.step.hi

0.00022613065326632098

In [5]:
file = "2d_map_parabola_zoom.txt"

"2d_map_parabola_zoom.txt"

In [9]:
for (index_I0, I0) in enumerate(rangeI)
    
        if index_I0 == 1
            global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
        end
        
        U0 = parabola(I0)
    
        print_output(index_I0, I0, U0, initial_condition_function)
    
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    
        ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_function = tr[end, :]
    
        println("last point: $initial_condition_function");flush(stdout)
    
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 4, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
    
        println("----------------------")
    
        if I0 != startI
            move_to_left( I0, U0, initial_condition_function, step_to_left )
        end
        
end

index: 1
I0: -1.75
U0: 0.3052996106902357
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [5.459768226577614, 0.8026888078499941, 0.6717356838694429, 0.08021679550830378]
LSE: [-0.0008047912203706438, -0.5555556779286381, -0.7737067863575887, -8.340035071510636]
----------------------
index: 2
I0: -1.7497738693467337
U0: 0.30523467409531024
initial condition: [5.459768226577614, 0.8026888078499941, 0.6717356838694429, 0.08021679550830378]
last point: [1.115210986675084, 0.6140677534468426, 0.7687180392541311, 0.07059125844503958]
LSE: [-0.0030210527397756975, -0.5555556345255644, -0.7745229306273523, -8.335971008827972]
----------------------

<<<<<<<<<<<<<<<<<<<<<
MOVE TO LEFT
 <- 
index: 1
I0: -1.75
U0: 0.30523467409531024
initial condition: [1.115210986675084, 0.6140677534468426, 0.7687180392541311, 0.07059125844503958]
last point: [3.2439442956916515, 0.4754290828209932, 0.7902197680653652, 0.07336065138620006]
LSE: [-0.0020095679802882437, -0.555555555638207, -0.76387825015749

last point: [1.17836128498117, 0.8187293520489513, 0.7084019010801327, 0.0686960027329488]
LSE: [0.0002957898048592721, -0.5555556412585043, -0.7659890148425734, -8.326069230111969]
_________________

 <- 
index: 2
I0: -1.7490954773869345
U0: 0.3049102001482039
initial condition: [1.17836128498117, 0.8187293520489513, 0.7084019010801327, 0.0686960027329488]
last point: [3.231764355685327, 0.8298397971633787, 0.6696921128141934, 0.07994585203646981]
LSE: [0.001633354727382591, -0.5555556439740159, -0.7549625017617597, -8.322956214448462]
_________________

 <- 
index: 3
I0: -1.749321608040201
U0: 0.3049102001482039
initial condition: [3.231764355685327, 0.8298397971633787, 0.6696921128141934, 0.07994585203646981]
last point: [4.145986030861164, 0.818278594251889, 0.6694383658181513, 0.08081019811625603]
LSE: [0.004123319104886987, -0.5555556458626659, -0.7438982853319757, -8.320521686457123]
_________________

 <- 
index: 4
I0: -1.7495477386934672
U0: 0.3049102001482039
initial conditio

_________________

index: 10
I0: -1.747964824120603
U0: 0.30471568300195706
initial condition: [0.916248257470461, 0.7248582631963619, 0.7435099838578649, 0.06811351069295199]
last point: [1.751369038928814, 0.8466380626791521, 0.6806265786970325, 0.0745686342942003]
LSE: [-0.005590078288841055, -0.5555556373817572, -0.779727470460581, -8.318141824069214]
----------------------

<<<<<<<<<<<<<<<<<<<<<
MOVE TO LEFT
 <- 
index: 1
I0: -1.7481909547738692
U0: 0.30471568300195706
initial condition: [1.751369038928814, 0.8466380626791521, 0.6806265786970325, 0.0745686342942003]
last point: [4.208916426328143, 0.4483281924611532, 0.7928998786147057, 0.07346200928477416]
LSE: [0.0001652805835168716, -0.5555555905974736, -0.7683645523325535, -8.317899929216395]
_________________

 <- 
index: 2
I0: -1.7484170854271355
U0: 0.30471568300195706
initial condition: [4.208916426328143, 0.4483281924611532, 0.7928998786147057, 0.07346200928477416]
last point: [0.939871845956364, 0.7475603888998773, 0.736

last point: [2.5986681392680855, 0.8382252363460791, 0.6711612336941191, 0.07847355052166001]
LSE: [0.0068529062073138915, -0.5555555809200493, -0.7698305746521646, -8.313006011845397]
_________________

 <- 
index: 2
I0: -1.747964824120603
U0: 0.3045860745802994
initial condition: [2.5986681392680855, 0.8382252363460791, 0.6711612336941191, 0.07847355052166001]
last point: [1.6244554031317142, 0.8461313888073732, 0.683622736954297, 0.073982598201892]
LSE: [0.004400948856184695, -0.5555556405778977, -0.758618142959983, -8.310345256010503]
_________________

 <- 
index: 3
I0: -1.7481909547738694
U0: 0.3045860745802994
initial condition: [1.6244554031317142, 0.8461313888073732, 0.683622736954297, 0.073982598201892]
last point: [0.9192284924598583, 0.6843468062526862, 0.7532787061682225, 0.06972452690089852]
LSE: [0.0006616749107714147, -0.5555557809693693, -0.7467394268948488, -8.310101535741952]
_________________

 <- 
index: 4
I0: -1.7484170854271357
U0: 0.3045860745802994
initial cond

_________________

 <- 
index: 12
I0: -1.7499999999999998
U0: 0.3045212912722227
initial condition: [0.9288800705978745, 0.7426389083191326, 0.7371070361954045, 0.07133755578319009]
last point: [1.7527547412637645, 0.8489830418936603, 0.6768603913456704, 0.07910607188460564]
LSE: [0.002236111634131526, -0.555555859070826, -0.631780135585088, -8.274622225927788]
_________________

index: 14
I0: -1.7470603015075377
U0: 0.30445652189931416
initial condition: [0.9631756328842095, 0.7599734360593523, 0.7328086993051823, 0.06737489823839271]
last point: [1.7410958051333154, 0.5463401200264062, 0.7798249592003671, 0.07122454806696162]
LSE: [0.0003694293943826894, -0.5555556286985354, -0.7819329364917104, -8.310035523704911]
----------------------

<<<<<<<<<<<<<<<<<<<<<
MOVE TO LEFT
 <- 
index: 1
I0: -1.747286432160804
U0: 0.30445652189931416
initial condition: [1.7410958051333154, 0.5463401200264062, 0.7798249592003671, 0.07122454806696162]
last point: [11.032617674066005, 0.7392317628443245,

...

I0: -1.7429899497487436
U0: 0.30200561168833095
initial condition: [1.4084570012388675, 0.8423322337724914, 0.6873448538855688, 0.07587799673373272]
last point: [4.159276907385164, 0.4546626822741455, 0.787508026397161, 0.0787857217723113]
LSE: [-0.0006269259015895372, -0.5370693546496539, -0.5565964599460568, -8.152289173279966]
_________________

 <- 
index: 21
I0: -1.7432160804020098
U0: 0.30200561168833095
initial condition: [4.159276907385164, 0.4546626822741455, 0.787508026397161, 0.0787857217723113]
last point: [1.839238537764802, 0.8495945846553506, 0.6709821665023837, 0.08181974906025251]
LSE: [0.01054263841499228, -0.5179212260572026, -0.5547405740974495, -8.1475677737121]
_________________

 <- 
index: 22
I0: -1.743442211055276
U0: 0.30200561168833095
initial condition: [1.839238537764802, 0.8495945846553506, 0.6709821665023837, 0.08181974906025251]
last point: [7.39681468138392, 0.40010692710030066, 0.790720887383759, 0.08110006562439546]
LSE: [0.0019359719404072934, -0.497

LSE: [-0.3182745765539577, -0.31699988682190117, -11.617559298461469, -15.081719949104325]
_________________

 <- 
index: 43
I0: -1.748190954773869
U0: 0.30200561168833095
initial condition: [0.9401320269281324, 0.9246725307323792, 0.5776773155959638, 0.48897368999127694]
last point: [0.9399752478987538, 0.9246779326186387, 0.5777288583513522, 0.4889937156725428]
LSE: [-0.32096253133983466, -0.31450534876200187, -11.615550452830227, -15.08495640046555]
_________________

 <- 
index: 44
I0: -1.7484170854271353
U0: 0.30200561168833095
initial condition: [0.9399752478987538, 0.9246779326186387, 0.5777288583513522, 0.4889937156725428]
last point: [0.9398184920369685, 0.9246833340469452, 0.5777804083832422, 0.48901373913283813]
LSE: [-0.3197662481821209, -0.3158953172560249, -11.613544387284247, -15.088190068643035]
_________________

 <- 
index: 45
I0: -1.7486432160804015
U0: 0.30200561168833095
initial condition: [0.9398184920369685, 0.9246833340469452, 0.5777804083832422, 0.4890137391328

U0: 0.30194138578697716
initial condition: [1.0144464453434725, 0.7805524567362181, 0.722474457145145, 0.06929655022276535]
last point: [0.9195011068728862, 0.7198033717463567, 0.7414149088005267, 0.0705215969398137]
LSE: [0.0020531572147578234, -0.5555555392220984, -0.6489681842503671, -8.194644851118342]
_________________

 <- 
index: 14
I0: -1.741407035175879
U0: 0.30194138578697716
initial condition: [0.9195011068728862, 0.7198033717463567, 0.7414149088005267, 0.0705215969398137]
last point: [0.9388259965957175, 0.7434505463800363, 0.7346281658248914, 0.07044988303927985]
LSE: [0.0012597971684346846, -0.5555557835143514, -0.6357181519637364, -8.189370420604789]
_________________

 <- 
index: 15
I0: -1.7416331658291453
U0: 0.30194138578697716
initial condition: [0.9388259965957175, 0.7434505463800363, 0.7346281658248914, 0.07044988303927985]
last point: [0.9732708430423955, 0.764691782755524, 0.7277848121734334, 0.07051779484399713]
LSE: [0.0024376865083871042, -0.5555556406400356, 

LSE: [0.07817182573048953, -0.0005115539955518099, -2.0792596210184264, -11.956047042628665]
_________________

 <- 
index: 36
I0: -1.7463819095477382
U0: 0.30194138578697716
initial condition: [1.656279982414843, 0.8576453621514244, 0.6613263358738246, 0.09743906203924876]
last point: [0.6567809864583983, 0.9586118216913957, 0.4390800622425247, 0.4579515374529983]
LSE: [0.07422476527097124, -0.0035965137591732, -2.454013551124074, -13.018542686213916]
_________________

 <- 
index: 37
I0: -1.7466080402010045
U0: 0.30194138578697716
initial condition: [0.6567809864583983, 0.9586118216913957, 0.4390800622425247, 0.4579515374529983]
last point: [2.294615640540181, 0.8473175592631309, 0.6531874376347125, 0.11346223256073737]
LSE: [0.1088778165245972, 0.008510334227026878, -2.698028489032389, -13.603433764203938]
_________________

 <- 
index: 38
I0: -1.7468341708542707
U0: 0.30194138578697716
initial condition: [2.294615640540181, 0.8473175592631309, 0.6531874376347125, 0.1134622325607373

Excessive output truncated after 524334 bytes.

In [8]:
function move_to_left( I0, U0, initial_condition_function, step_to_left )
    
    println("");flush(stdout)
    println("<<<<<<<<<<<<<<<<<<<<<");flush(stdout)
    
    println("MOVE TO LEFT")
        
    range_to_left_border = range( I0 - step_to_left, startI, step = -step_to_left )
        
    for (index_left, I0_left) in enumerate( range_to_left_border )
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
        end
            
        println(" <- ")
        print_output(index_left, I0_left, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0_left $U0 $(λλ[1])")
        end
        separate()
    end
end

move_to_left (generic function with 1 method)